In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

plt.rcParams['font.sans-serif'] = ['SimHei']#Windows系统常用黑体
plt.rcParams['axes.unicode_minus'] = False#正常显示负号

data = pd.read_csv('data.csv')

#先筛选字符串变量
discrete_features = data.select_dtypes(include=['object']).columns.tolist()
#Home Ownership标签编码
home_ownership_mapping = {
    'Omn Home': 1, 
    'Rent': 2, 
    'Have Mortgage': 3,
    'Have Mortgage': 4,
}
data['Home Ownership'] = data['Home Ownership'].map(home_ownership_mapping)


#Year in current job 标签编码

years_in_job_mappings = {
    '< 1 year': 1,
    '1 year': 2,
    '2 years': 3,
    '3 years': 4,
    '4 years': 5,
    '5 years': 6,
    '6 years': 7,
    '7 years': 8,
    '8 years': 9,
    '9 years': 10,
    '10+ years': 11,
}
data['Years in current job'] = data['Years in current job'].map(years_in_job_mappings)


#Purpose 独热编码，记得需要将bool类型转换为数值

data = pd.get_dummies(data, columns=['Purpose'])
data2 = pd.read_csv('data.csv')#重新读取数据，用来做列名对比
list_final = []#新建一个空列表，用于存放独热编码后的新增特征名
for i in data.columns:
    if i not in data2.columns:
        list_final.append(i)#打印出来的就是独热编码后的新增特征名
for i in list_final:
    data[i] = data[i].astype(int)#i就是独热编码后的新增特征名，将其转换为数值型态

# Term 0-1映射
term_mapping = {
    'Short Term': 0,
    'Long Term': 1
}
data['Term'] = data['Term'].map(term_mapping)
data.rename(columns={'Term': 'Long Term'}, inplace=True) # 重命名列
continuous_features = data.select_dtypes(include=['int64', 'float64']).columns.tolist()  #把筛选出来的列名转换成列表
 
 # 连续特征用中位数补全
for feature in continuous_features:     
    mode_value = data[feature].mode()[0]            #获取该列的众数。
    data[feature].fillna(mode_value, inplace=True)          #用众数填充该列的缺失值，inplace=True表示直接在原数据上修改。


from sklearn.model_selection import train_test_split
X = data.drop(['Credit Default'], axis=1)  # 特征，axis=1表示按列删除
y = data['Credit Default'] # 标签
# # 按照8:2划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)  # 80%训练集，20%测试集
# 很多调参函数自带交叉验证，甚至是必选的参数，你如果想要不交叉反而实现起来会麻烦很多
# 所以这里我们还是只划分一次数据集


In [14]:
from sklearn.ensemble import RandomForestClassifier #随机森林分类器

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score # 用于评估分类器性能的指标
from sklearn.metrics import classification_report, confusion_matrix #用于生成分类报告和混淆矩阵
import warnings #用于忽略警告信息
warnings.filterwarnings("ignore") # 忽略所有警告信息
# --- 1. 默认参数的随机森林 ---
# 评估基准模型，这里确实不需要验证集
print("--- 1. 默认参数随机森林 (训练集 -> 测试集) ---")
import time # 这里介绍一个新的库，time库，主要用于时间相关的操作，因为调参需要很长时间，记录下会帮助后人知道大概的时长
start_time = time.time() # 记录开始时间
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train) # 在训练集上训练
rf_pred = rf_model.predict(X_test) # 在测试集上预测
end_time = time.time() # 记录结束时间

print(f"训练与预测耗时: {end_time - start_time:.4f} 秒")
print("\n默认随机森林 在测试集上的分类报告：")
print(classification_report(y_test, rf_pred))
print("默认随机森林 在测试集上的混淆矩阵：")
print(confusion_matrix(y_test, rf_pred))

--- 1. 默认参数随机森林 (训练集 -> 测试集) ---
训练与预测耗时: 0.8469 秒

默认随机森林 在测试集上的分类报告：
              precision    recall  f1-score   support

           0       0.77      0.97      0.86      1059
           1       0.80      0.29      0.43       441

    accuracy                           0.77      1500
   macro avg       0.78      0.63      0.64      1500
weighted avg       0.77      0.77      0.73      1500

默认随机森林 在测试集上的混淆矩阵：
[[1026   33]
 [ 313  128]]


In [15]:
# 打印标题，表明这是方差筛选的部分
print("--- 方差筛选 (Variance Threshold) ---")

# 导入需要的工具库
from sklearn.feature_selection import VarianceThreshold  # 方差筛选工具，用于剔除方差小的特征
import time  # 用于记录代码运行时间，方便比较效率

# 记录开始时间，后面会计算整个过程耗时
start_time = time.time()

# 创建方差筛选器，设置方差阈值为0.01
# 阈值是指方差的最小值，低于这个值的特征会被删除（可以根据数据情况调整阈值）
selector = VarianceThreshold(threshold=0.01)

# 对训练数据进行方差筛选，fit_transform会计算每个特征的方差并剔除不满足阈值的特征
# X_train是原始训练数据，X_train_var是筛选后的训练数据
X_train_var = selector.fit_transform(X_train)

# 对测试数据应用同样的筛选规则，transform会直接用训练数据的筛选结果处理测试数据
# X_test是原始测试数据，X_test_var是筛选后的测试数据
X_test_var = selector.transform(X_test)

# 获取被保留下来的特征名称
# selector.get_support()返回一个布尔值列表，表示哪些特征被保留，这个是selector这个实例化的类的一个方法
# X_train.columns是特征的名称，结合布尔值列表可以提取保留特征的名字
selected_features_var = X_train.columns[selector.get_support()].tolist()

# 打印筛选后保留的特征数量和具体特征名称，方便查看结果
print(f"方差筛选后保留的特征数量: {len(selected_features_var)}")
print(f"保留的特征: {selected_features_var}")

# 创建一个随机森林分类模型，用于在筛选后的数据上进行训练和预测
# random_state=42是为了保证每次运行结果一致，方便教学和对比
rf_model_var = RandomForestClassifier(random_state=42)

# 在筛选后的训练数据上训练模型
# X_train_var是筛选后的特征数据，y_train是对应的目标标签
rf_model_var.fit(X_train_var, y_train)

# 使用训练好的模型对筛选后的测试数据进行预测
# X_test_var是筛选后的测试特征数据，rf_pred_var是预测结果
rf_pred_var = rf_model_var.predict(X_test_var)

# 记录结束时间，计算整个训练和预测过程的耗时
end_time = time.time()
print(f"训练与预测耗时: {end_time - start_time:.4f} 秒")

# 打印模型在测试集上的分类报告，展示模型的性能
# 分类报告包括精确率、召回率、F1分数等指标，帮助评估模型好坏
print("\n方差筛选后随机森林在测试集上的分类报告：")
print(classification_report(y_test, rf_pred_var))

# 打印混淆矩阵，展示模型预测的详细结果
# 混淆矩阵显示了真实标签和预测标签的对应情况，比如多少样本被正确分类，多少被错分
print("方差筛选后随机森林在测试集上的混淆矩阵：")
print(confusion_matrix(y_test, rf_pred_var))

--- 方差筛选 (Variance Threshold) ---
方差筛选后保留的特征数量: 20
保留的特征: ['Id', 'Annual Income', 'Years in current job', 'Tax Liens', 'Number of Open Accounts', 'Years of Credit History', 'Maximum Open Credit', 'Number of Credit Problems', 'Months since last delinquent', 'Bankruptcies', 'Long Term', 'Current Loan Amount', 'Current Credit Balance', 'Monthly Debt', 'Credit Score', 'Purpose_business loan', 'Purpose_buy a car', 'Purpose_debt consolidation', 'Purpose_home improvements', 'Purpose_other']
训练与预测耗时: 0.9409 秒

方差筛选后随机森林在测试集上的分类报告：
              precision    recall  f1-score   support

           0       0.77      0.96      0.85      1059
           1       0.75      0.30      0.43       441

    accuracy                           0.76      1500
   macro avg       0.76      0.63      0.64      1500
weighted avg       0.76      0.76      0.73      1500

方差筛选后随机森林在测试集上的混淆矩阵：
[[1014   45]
 [ 308  133]]


In [16]:
print("--- 皮尔逊相关系数筛选 ---")
from sklearn.feature_selection import SelectKBest, f_classif
import time

start_time = time.time()

# 计算特征与目标变量的相关性，选择前k个特征（这里设为10个，可调整）
# 注意：皮尔逊相关系数通常用于回归问题（连续型目标变量），但如果目标是分类问题，可以用f_classif
k = 10
selector = SelectKBest(score_func=f_classif, k=k)
X_train_corr = selector.fit_transform(X_train, y_train)
X_test_corr = selector.transform(X_test)

# 获取筛选后的特征名
selected_features_corr = X_train.columns[selector.get_support()].tolist()
print(f"皮尔逊相关系数筛选后保留的特征数量: {len(selected_features_corr)}")
print(f"保留的特征: {selected_features_corr}")

# 训练随机森林模型
rf_model_corr = RandomForestClassifier(random_state=42)
rf_model_corr.fit(X_train_corr, y_train)
rf_pred_corr = rf_model_corr.predict(X_test_corr)

end_time = time.time()
print(f"训练与预测耗时: {end_time - start_time:.4f} 秒")
print("\n皮尔逊相关系数筛选后随机森林在测试集上的分类报告：")
print(classification_report(y_test, rf_pred_corr))
print("皮尔逊相关系数筛选后随机森林在测试集上的混淆矩阵：")
print(confusion_matrix(y_test, rf_pred_corr))

--- 皮尔逊相关系数筛选 ---
皮尔逊相关系数筛选后保留的特征数量: 10
保留的特征: ['Id', 'Annual Income', 'Tax Liens', 'Number of Open Accounts', 'Number of Credit Problems', 'Long Term', 'Current Loan Amount', 'Credit Score', 'Purpose_business loan', 'Purpose_small business']
训练与预测耗时: 0.7471 秒

皮尔逊相关系数筛选后随机森林在测试集上的分类报告：
              precision    recall  f1-score   support

           0       0.76      0.94      0.84      1059
           1       0.67      0.31      0.42       441

    accuracy                           0.75      1500
   macro avg       0.72      0.62      0.63      1500
weighted avg       0.74      0.75      0.72      1500

皮尔逊相关系数筛选后随机森林在测试集上的混淆矩阵：
[[992  67]
 [306 135]]


In [17]:
print("--- Lasso筛选 (L1正则化) ---")
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel
import time

start_time = time.time()

# 使用Lasso回归进行特征筛选
lasso = Lasso(alpha=0.01, random_state=42)  # alpha值可调整
selector = SelectFromModel(lasso)
selector.fit(X_train, y_train)
X_train_lasso = selector.transform(X_train)
X_test_lasso = selector.transform(X_test)

# 获取筛选后的特征名
selected_features_lasso = X_train.columns[selector.get_support()].tolist()
print(f"Lasso筛选后保留的特征数量: {len(selected_features_lasso)}")
print(f"保留的特征: {selected_features_lasso}")

# 训练随机森林模型
rf_model_lasso = RandomForestClassifier(random_state=42)
rf_model_lasso.fit(X_train_lasso, y_train)
rf_pred_lasso = rf_model_lasso.predict(X_test_lasso)

end_time = time.time()
print(f"训练与预测耗时: {end_time - start_time:.4f} 秒")
print("\nLasso筛选后随机森林在测试集上的分类报告：")
print(classification_report(y_test, rf_pred_lasso))
print("Lasso筛选后随机森林在测试集上的混淆矩阵：")
print(confusion_matrix(y_test, rf_pred_lasso))

--- Lasso筛选 (L1正则化) ---
Lasso筛选后保留的特征数量: 6
保留的特征: ['Years in current job', 'Number of Open Accounts', 'Years of Credit History', 'Months since last delinquent', 'Long Term', 'Credit Score']
训练与预测耗时: 0.4751 秒

Lasso筛选后随机森林在测试集上的分类报告：
              precision    recall  f1-score   support

           0       0.77      0.94      0.84      1059
           1       0.68      0.32      0.44       441

    accuracy                           0.76      1500
   macro avg       0.72      0.63      0.64      1500
weighted avg       0.74      0.76      0.72      1500

Lasso筛选后随机森林在测试集上的混淆矩阵：
[[992  67]
 [299 142]]


In [18]:
print("--- 树模型自带的重要性筛选 ---")
from sklearn.feature_selection import SelectFromModel
import time

start_time = time.time()

# 使用随机森林的特征重要性进行筛选
rf_selector = RandomForestClassifier(random_state=42)
rf_selector.fit(X_train, y_train)
selector = SelectFromModel(rf_selector, threshold="mean")  # 阈值设为平均重要性，可调整
X_train_rf = selector.transform(X_train)
X_test_rf = selector.transform(X_test)

# 获取筛选后的特征名
selected_features_rf = X_train.columns[selector.get_support()].tolist()
print(f"树模型重要性筛选后保留的特征数量: {len(selected_features_rf)}")
print(f"保留的特征: {selected_features_rf}")

# 训练随机森林模型
rf_model_rf = RandomForestClassifier(random_state=42)
rf_model_rf.fit(X_train_rf, y_train)
rf_pred_rf = rf_model_rf.predict(X_test_rf)

end_time = time.time()
print(f"训练与预测耗时: {end_time - start_time:.4f} 秒")
print("\n树模型重要性筛选后随机森林在测试集上的分类报告：")
print(classification_report(y_test, rf_pred_rf))
print("树模型重要性筛选后随机森林在测试集上的混淆矩阵：")
print(confusion_matrix(y_test, rf_pred_rf))

--- 树模型自带的重要性筛选 ---
树模型重要性筛选后保留的特征数量: 11
保留的特征: ['Id', 'Annual Income', 'Years in current job', 'Number of Open Accounts', 'Years of Credit History', 'Maximum Open Credit', 'Months since last delinquent', 'Current Loan Amount', 'Current Credit Balance', 'Monthly Debt', 'Credit Score']
训练与预测耗时: 1.9563 秒

树模型重要性筛选后随机森林在测试集上的分类报告：
              precision    recall  f1-score   support

           0       0.76      0.97      0.85      1059
           1       0.79      0.27      0.40       441

    accuracy                           0.76      1500
   macro avg       0.78      0.62      0.63      1500
weighted avg       0.77      0.76      0.72      1500

树模型重要性筛选后随机森林在测试集上的混淆矩阵：
[[1027   32]
 [ 321  120]]


In [19]:
print("--- SHAP重要性筛选 ---")
import shap
from sklearn.feature_selection import SelectKBest
import time

start_time = time.time()

# 使用随机森林模型计算SHAP值
rf_shap = RandomForestClassifier(random_state=42)
rf_shap.fit(X_train, y_train)
explainer = shap.TreeExplainer(rf_shap)
shap_values = explainer.shap_values(X_train)

# 计算每个特征的平均SHAP值（取绝对值的平均）
mean_shap = np.abs(shap_values[1]).mean(axis=0)  # shap_values[1]对应正类
k = 10  # 选择前10个特征，可调整
top_k_indices = np.argsort(mean_shap)[-k:]
X_train_shap = X_train.iloc[:, top_k_indices]
X_test_shap = X_test.iloc[:, top_k_indices]

# 获取筛选后的特征名
selected_features_shap = X_train.columns[top_k_indices].tolist()
print(f"SHAP重要性筛选后保留的特征数量: {len(selected_features_shap)}")
print(f"保留的特征: {selected_features_shap}")

# 训练随机森林模型
rf_model_shap = RandomForestClassifier(random_state=42)
rf_model_shap.fit(X_train_shap, y_train)
rf_pred_shap = rf_model_shap.predict(X_test_shap)

end_time = time.time()
print(f"训练与预测耗时: {end_time - start_time:.4f} 秒")
print("\nSHAP重要性筛选后随机森林在测试集上的分类报告：")
print(classification_report(y_test, rf_pred_shap))
print("SHAP重要性筛选后随机森林在测试集上的混淆矩阵：")
print(confusion_matrix(y_test, rf_pred_shap))

--- SHAP重要性筛选 ---
SHAP重要性筛选后保留的特征数量: 2
保留的特征: ['Id', 'Home Ownership']
训练与预测耗时: 258.0102 秒

SHAP重要性筛选后随机森林在测试集上的分类报告：
              precision    recall  f1-score   support

           0       0.71      0.73      0.72      1059
           1       0.30      0.28      0.29       441

    accuracy                           0.60      1500
   macro avg       0.50      0.50      0.50      1500
weighted avg       0.59      0.60      0.59      1500

SHAP重要性筛选后随机森林在测试集上的混淆矩阵：
[[775 284]
 [319 122]]


In [ ]:
print("--- 递归特征消除 (RFE) ---")
from sklearn.feature_selection import RFE
import time

start_time = time.time()

# 使用随机森林作为基础模型进行RFE
base_model = RandomForestClassifier(random_state=42)
rfe = RFE(base_model, n_features_to_select=10)  # 选择10个特征，可调整
rfe.fit(X_train, y_train)
X_train_rfe = rfe.transform(X_train)
X_test_rfe = rfe.transform(X_test)

# 获取筛选后的特征名
selected_features_rfe = X_train.columns[rfe.support_].tolist()
print(f"RFE筛选后保留的特征数量: {len(selected_features_rfe)}")
print(f"保留的特征: {selected_features_rfe}")

# 训练随机森林模型
rf_model_rfe = RandomForestClassifier(random_state=42)
rf_model_rfe.fit(X_train_rfe, y_train)
rf_pred_rfe = rf_model_rfe.predict(X_test_rfe)

end_time = time.time()
print(f"训练与预测耗时: {end_time - start_time:.4f} 秒")
print("\nRFE筛选后随机森林在测试集上的分类报告：")
print(classification_report(y_test, rf_pred_rfe))
print("RFE筛选后随机森林在测试集上的混淆矩阵：")
print(confusion_matrix(y_test, rf_pred_rfe))